In [3]:
from qiskit import *
from qiskit.quantum_info import SparsePauliOp, Statevector

import numpy as np
from qiskit import QuantumRegister, QuantumCircuit

from math import cos, sin, cosh, sinh, atan, exp, pi
from scipy.optimize import minimize

import sys

import copy
import pickle

# from qubo_hamiltonian import *

import itertools

In [11]:
def partition_N(N:int):
    '''do the partition of a complete graph with N vertex, to find the optimal orders for edges to run circuit in parallel
    Args:
        N: number of qubits
    Return:
        pairs_all: list of qubit index pairs (edges) in a order to parallel the circuit
    '''
    indexs = range(N)
    pairs_all = []  

    ## swap indexes of even layer [0,1,2,3,4] -> [1,0,3,2,4]
    swap_even = [i + pow(-1, i) for i in range(N - (N%2))]  
    if (N%2) == 1:
        swap_even.append(N-1)
    ## swap indexes of even layer [0,1,2,3,4] -> [0,2,1,4,3]
    swap_odd = [0]
    swap_odd.extend([i + pow(-1, i+1) for i in range(1,N-(N+1)%2)])
    if (N%2) == 0:
        swap_odd.append(N-1)
    
    ## qubit pairs need to be implemented in layer 0
    pairs_even = [(i, i+1) for i in range(0, N-1, 2)]  
    pairs_all.append(pairs_even)
    indexs = np.array(indexs)[swap_even]   ### indexs after swap even

    for i in range(1, N):
        if (i%2)==1: ## odd layer
            pair_odd = [(indexs[i], indexs[i+1]) for i in range(1, N-1, 2)]
            pairs_all.append(pair_odd)
            indexs = np.array(indexs)[swap_odd]   ### indexs after swap odd

        elif (i%2)==0: ## even layer
            pair_even = [(indexs[i], indexs[i+1]) for i in range(0, N-1, 2)]
            pairs_all.append(pair_even)
            indexs = np.array(indexs)[swap_even]   ### indexs after swap even

    return pairs_all

In [13]:
def find_light_cone(pairs):
    lightcone_dict = {}
    for index, list in enumerate(pairs):
        for pair in list:
            qi, qj = pair
            relevent_pairs = []  ##  qubit pairs in the previous layer that in the lightcone of the current pair
            if index > 0:
                for pair_layerm1 in pairs[index-1]: ## qubit pairs in the previous layer
                    if (qi in pair_layerm1) or (qj in pair_layerm1):
                        relevent_pairs.append(pair_layerm1)
            lightcone_dict[pair] = relevent_pairs
    return lightcone_dict

In [3]:
def circuit_update_zz(Edge,State, Tauc, N):
    i = min(Edge)
    j = max(Edge)

    zzop = SparsePauliOp.from_sparse_list([('ZZ', [j, i], -sinh(Tauc) )], N)
    zzop += SparsePauliOp.from_list([("I"*N, cosh(Tauc))])
    
    State = State.evolve(zzop)

    # Normalize the state vector manually
    norm = np.linalg.norm(State.data)
    State = State / norm

    # Print the normalized state vector
    # print("Normalized state:", State)

    return State

In [4]:
def circuit_update_theta(Edge, State, paras, N):
    
    j = min(Edge)    #I SWAPPED i AND j TO MATCH WITH YAHUI CIRCUIT
    i = max(Edge)

    zyop = SparsePauliOp.from_sparse_list([('ZY', [j, i], -1j*sin(paras[0]/2))], N)
    zyop += SparsePauliOp.from_list([("I"*N, cos(paras[0]/2))])

    yzop = SparsePauliOp.from_sparse_list([('YZ', [j, i], -1j*sin(paras[1]/2))], N)
    yzop += SparsePauliOp.from_list([("I"*N, cos(paras[1]/2))])

    op = zyop.compose(yzop)
    
    State = State.evolve(op)

    # print('states2', State)

    return State

In [5]:
def circuit_update_theta_Yahui(Edge, Circ, paras):

    qcirc = Circ.copy()
    
    i = min(Edge)
    j = max(Edge)

    ### exp{-i/2 ( params[2]*ZiYj + params[3]*YiZj )}
    qcirc.rx(-np.pi/2, i)
    qcirc.rz(-np.pi/2, j)

    qcirc.cx(i, j)
    qcirc.ry(paras[0], i)
    qcirc.rz(-paras[1], j)
    qcirc.cx(i, j)

    qcirc.rx(np.pi/2, i)
    qcirc.rz(np.pi/2, j)
    
    State = Statevector(qcirc)

    # print('states2 yahui', State)

    return State

In [6]:
def square_modulus_cost(Paras : list, *args):

    Edge = args[0]
    State = args[1]
    Tauc = args[2]
    N = args[3]

    State_zz = circuit_update_zz(Edge,State, Tauc, N)
    State_theta = circuit_update_theta(Edge,State, Paras, N)

    # Compute the scalar product (inner product) between the two state vectors
    inner_product = State_zz.inner(State_theta)

    # Compute the square modulus of the inner product
    square_modulus = abs(inner_product)**2

    # Print the inner product and its square modulus
    # print("Inner product:", inner_product)
    # print("Square modulus of the inner product:", square_modulus)

    return - square_modulus

In [7]:
def square_modulus_cost_Yahui(Paras : list, *args):

    Edge = args[0]
    Circ = args[1]
    Tauc = args[2]
    N = args[3]

    State =  Statevector(Circ)

    State_zz = circuit_update_zz(Edge,State, Tauc, N)
    State_theta = circuit_update_theta_Yahui(Edge, Circ, Paras)

    # Compute the scalar product (inner product) between the two state vectors
    inner_product = State_zz.inner(State_theta)

    # Compute the square modulus of the inner product
    square_modulus = abs(inner_product)**2

    # Print the inner product and its square modulus
    # print("Inner product:", inner_product)
    print("Square modulus of the inner product:", square_modulus)

    return - square_modulus

In [5]:
def square_modulus_cost_light_cone(Paras : list, *args):

    # para_init = np.zeros((len(lightcone_dict[edge]) + 1, 2))

    Edge_list = args[0]
    State = args[1]
    Tauc = args[2]
    Updated_state = args[3]
    N = args[4]

    State_zz = circuit_update_zz(Edge_list[-1], Updated_state, Tauc, N)
    
    for index, edge in enumerate(Edge_list):
        Parameters = [Paras[2*index], Paras[2*index + 1]]
        State = circuit_update_theta(edge,State, Parameters, N)

    # Compute the scalar product (inner product) between the two state vectors
    inner_product = State_zz.inner(State)

    # Compute the square modulus of the inner product
    square_modulus = abs(inner_product)**2

    # Print the inner product and its square modulus
    # print("Inner product:", inner_product)
    # print("Square modulus of the inner product:", square_modulus)

    return - square_modulus

In [6]:
def warm_start_parameters_lightcone(N : int, tau:float, edge_coeff_dict : dict, edges_columns :list,  eigen_list:list, lightcone_dict: dict):    

    eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
    edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
    exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

    q = QuantumRegister(N, name = 'q')
    circ = QuantumCircuit(q)
    circ.clear()
    circ.h(q[::])

    tau = 0.2

    # Z term
    for i in range(N):

        #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
        tauc = tau * edge_coeff_dict[(i,)] 
        para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
        edge_params_dict[(i,)] = para
        circ.ry(para, i)
        
    ## ZZ term 
    state = Statevector(circ)
    updated_state = state 

    print('\nnumber of columns is:', len(edges_columns))

    for column_index, column in enumerate(edges_columns):
        print('\n##################################################')
        print('column index', column_index, 'column', column)

        if column_index == 0: 

            first_column_state = state

            for edge in column:
                
                if len(lightcone_dict[edge]) != 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    sys.exit()
            
                print('\nedge', edge)  

                tauc = tau * edge_coeff_dict[edge]

                para_init = [0,0]

                final = minimize(square_modulus_cost,
                                    para_init,
                                    args = (edge, first_column_state, tauc, N),
                                    jac=False,
                                    bounds=None,
                                    method='L-BFGS-B',
                                    callback=None,
                                    options={'maxiter': 1000})

                para = final.x

                # print('opt paramenters', para)
                # print('final square modulus', final.fun)

                edge_params_dict[edge] = para
                # print('edge_params_dict:', edge_params_dict)

                # print('old state', state)
                first_column_state = circuit_update_theta(edge, first_column_state, para, N)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT

                # print('first column state - new state', first_column_state)
            
        else:
            
            for edge in column: 
                if len(lightcone_dict[edge]) == 0:
                        sys.stderr.write('something is wrong with the lightcones')
                        sys.exit()

                print('\nedge', edge)  

                # print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

                # print('initial state', state)

                # updated_state = copy.deepcopy(state)
                
                for old_edge in lightcone_dict[edge]:
                    # print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                    updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N)    
                
                # print('updated_state', updated_state)

                edge_list = lightcone_dict[edge] + [edge]
                # print('edge list', edge_list)
                
                tauc = tau * edge_coeff_dict[edge]

                para_init = np.zeros(2 + 2*len(lightcone_dict[edge]))
                # print('para init', para_init)

                final = minimize(square_modulus_cost_light_cone,
                        para_init,
                        args = (edge_list, state, tauc, updated_state, N),
                        jac=False,
                        bounds=None,
                        method='L-BFGS-B',
                        callback=None,
                        options={'maxiter': 1000})

                para = final.x
                # print('opt paramenters', para)
                # print('final square modulus', final.fun)

                para = (np.array(para)).reshape(-1, 2)
                # print('param list reshaped', para)
                for index, edge in enumerate(edge_list):
                    edge_params_dict[edge] = para[index] 
                # print('edge_params_dict', edge_params_dict)
            
            # print('\n######### previous column update ########')
            # print('column index', column_index, 'column', column)
            # print('previous column is:', edges_columns[column_index -1])

            for edge in edges_columns[column_index -1]:
                # print('edge', edge, 'parameter', edge_params_dict[edge])
                state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

            # print('circuit updated ad the previous column i.e. column', column_index -1)
            # print('updated statevector', state)

    # print('\n######### last column update ########')
    # print('last column is:', edges_columns[ -1])

    for edge in edges_columns[ -1]:
        # print('edge', edge, 'parameter', edge_params_dict[edge])
        state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

    # print('circuit updated at the last column i.e.', len(edges_columns) - 1)
    print('updated statevector', state)

    #generate params_list
    values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
    # Concatenate and flatten all arrays into a single array
    flattened_array = np.concatenate(values_as_arrays)
    # Convert the flattened array to a list if needed
    params_list = flattened_array.tolist()
    # print(' params_list' , params_list )

    state = np.array(state)

    for id in eigens_ids:
        eigen = eigen_list[id]
        poss = abs(state[id])**2
        # print('eigen', eigen, 'poss', poss)
        exp_poss_dict[eigen] = poss

    return edge_params_dict, params_list, exp_poss_dict

In [10]:
# def warm_start_parameters_lightcone(N : int, tau:float, edge_coeff_dict : dict, edges_columns :list, lightcone_dict: dict):    
    
#     edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer

#     q = QuantumRegister(N, name = 'q')
#     circ = QuantumCircuit(q)
#     circ.clear()
#     circ.h(q[::])

#     tau = 0.2

#     # Z term
#     for i in range(N):

#         #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
#         tauc = tau * edge_coeff_dict[(i,)] 
#         para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
#         edge_params_dict[(i,)] = para
#         circ.ry(para, i)
        
#     ## ZZ term 
#     state = Statevector(circ)
#     updated_state = state 

#     print('\nnumber of columns is:', len(edges_columns))

#     for column_index, column in enumerate(edges_columns):
#         print('\n##################################################')
#         print('column index', column_index, 'column', column)

#         if column_index == 0: 

#             first_column_state = state

#             for edge in column:
                
#                 if len(lightcone_dict[edge]) != 0:
#                     sys.stderr.write('something is wrong with the lightcones')
#                     sys.exit()
            
#                 print('\nedge', edge)  

#                 tauc = tau * edge_coeff_dict[edge]

#                 para_init = [0,0]

#                 final = minimize(square_modulus_cost,
#                                     para_init,
#                                     args = (edge, first_column_state, tauc),
#                                     jac=False,
#                                     bounds=None,
#                                     method='L-BFGS-B',
#                                     callback=None,
#                                     options={'maxiter': 100})

#                 para = final.x

#                 print('opt paramenters', para)
#                 print('final square modulus', final.fun)

#                 edge_params_dict[edge] = para
#                 print('edge_params_dict:', edge_params_dict)

#                 print('old state', state)
#                 first_column_state = circuit_update_theta(edge, first_column_state, para)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT

#                 print('first column state - new state', first_column_state)
            
#         else:
            
#             for edge in column: 
#                 if len(lightcone_dict[edge]) == 0:
#                         sys.stderr.write('something is wrong with the lightcones')
#                         sys.exit()

#                 print('\nedge', edge)  

#                 print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

#                 print('initial state', state)

#                 updated_state = copy.deepcopy(state)
                
#                 for old_edge in lightcone_dict[edge]:
#                     print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
#                     updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge])    
                
#                 print('updated_state', updated_state)

#                 edge_list = lightcone_dict[edge] + [edge]
#                 print('edge list', edge_list)
                
#                 tauc = tau * edge_coeff_dict[edge]

#                 para_init = np.zeros(2 + 2*len(lightcone_dict[edge]))
#                 print('para init', para_init)

#                 final = minimize(square_modulus_cost_light_cone,
#                         para_init,
#                         args = (edge_list, state, tauc, updated_state),
#                         jac=False,
#                         bounds=None,
#                         method='L-BFGS-B',
#                         callback=None,
#                         options={'maxiter': 100})

#                 para = final.x
#                 print('opt paramenters', para)
#                 print('final square modulus', final.fun)

#                 para = (np.array(para)).reshape(-1, 2)
#                 print('param list reshaped', para)
#                 for index, edge in enumerate(edge_list):
#                     edge_params_dict[edge] = para[index] 
#                 print('edge_params_dict', edge_params_dict)
            
#             print('\n######### previous column update ########')
#             print('column index', column_index, 'column', column)
#             print('previous column is:', edges_columns[column_index -1])

#             for edge in edges_columns[column_index -1]:
#                 print('edge', edge, 'parameter', edge_params_dict[edge])
#                 state = circuit_update_theta(edge, state, edge_params_dict[edge])

#             print('circuit updated ad the previous column i.e. column', column_index -1)
#             print('updated statevector', state)

#     print('\n######### last column update ########')
#     print('last column is:', edges_columns[ -1])

#     for edge in edges_columns[ -1]:
#         print('edge', edge, 'parameter', edge_params_dict[edge])
#         state = circuit_update_theta(edge, state, edge_params_dict[edge])

#     print('circuit updated at the last column i.e.', len(edges_columns) - 1)
#     print('updated statevector', state)

#     #generate params_list
#     values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
#     # Concatenate and flatten all arrays into a single array
#     flattened_array = np.concatenate(values_as_arrays)
#     # Convert the flattened array to a list if needed
#     params_list = flattened_array.tolist()
#     print(' params_list' , params_list )

#     return edge_params_dict, params_list

In [7]:
def Hamiltonian_qubo(N, edge_list, h_list, J_list):
    """Hamiltonian defined by a N vertex graph with connected edge in edge_list
    Args:
        N: number of qubits
        edge_list: list of edges(qubit index pairs)
        h_list: coefficients of single Pauli Z term
        J_list: coefficients of ZZ term
    Return:
        H: PauliSumOp, Hamiltonian

    """
    pauli_list = []
    for i in range(N):
        pauli_str = (N-i-1)*'I' + 'Z' + i*'I'
        op = Pauli(pauli_str)
        pauli_list.append((op.to_label(), h_list[i]))
        
    for k, (i, j) in enumerate(edge_list):
        x_p = np.zeros(N, dtype = bool)
        z_p = np.zeros(N, dtype = bool)
        z_p[i] = True
        z_p[j] = True
        op = Pauli((z_p, x_p))
        pauli_list.append((op.to_label(), J_list[k]))
        
    H = PauliSumOp.from_list(pauli_list)
    
    return H

In [14]:
N = 6
tau = 0.2
r = 0

edges_columns = partition_N(N)
print('edges_columns', edges_columns)
lightcone_dict = find_light_cone(edges_columns)
print('lightcone_dict', lightcone_dict)

#region load qubo instances, get Hamiltonian and edge_coeff_dict
instance_dir = '../instances/complete/N_' + str(N )
with open(instance_dir + '/QUBO_' + str(N ) + 'V_comp_'+ '.gpickle', 'rb') as f:
    G = pickle.load(f)
edge_list = G.edges()
pairs_all = list(itertools.chain.from_iterable(partition_N(N)))
print('pairs_all', pairs_all)
print('edge_list', edge_list)
coeff_list = np.loadtxt(instance_dir + '/QUBO_coeff_' + str(N) + 'V_comp_'+ 'r_'+ str(r)+ '.txt')
h_list = coeff_list[: N ]
J_list = coeff_list[N :]

#extrcact Hamiltonian, edge coefficients and eigen list
H = Hamiltonian_qubo(N, edge_list, h_list, J_list)
eigen_list = H.to_spmatrix().diagonal().real
print('Hamiltonian', H)
print('eigen list', eigen_list[:13])
print('eigen list sorted',  np.sort(eigen_list))

edges_columns [[(0, 1), (2, 3), (4, 5)], [(0, 3), (2, 5)], [(1, 3), (0, 5), (2, 4)], [(1, 5), (0, 4)], [(3, 5), (1, 4), (0, 2)], [(3, 4), (1, 2)]]
lightcone_dict {(0, 1): [], (2, 3): [], (4, 5): [], (0, 3): [(0, 1), (2, 3)], (2, 5): [(2, 3), (4, 5)], (1, 3): [(0, 3)], (0, 5): [(0, 3), (2, 5)], (2, 4): [(2, 5)], (1, 5): [(1, 3), (0, 5)], (0, 4): [(0, 5), (2, 4)], (3, 5): [(1, 5)], (1, 4): [(1, 5), (0, 4)], (0, 2): [(0, 4)], (3, 4): [(3, 5), (1, 4)], (1, 2): [(1, 4), (0, 2)]}


FileNotFoundError: [Errno 2] No such file or directory: '../instances/complete/N_6/QUBO_6V_comp_.gpickle'

In [12]:
# Initialize dictionary for single Pauli Z term with coefficient from h_list
Edge_coeff_dict = {}
Edge_coeff_dict.update({(i,): h_val for i, h_val in enumerate(h_list)})
#Initialize dictionary for Pauli ZZ term with coefficient from from J_list
Edge_coeff_dict.update({edge: J_val for edge, J_val in zip(edge_list, J_list)})
print('edge_coeff_dict' , Edge_coeff_dict)
#endregion

edge_coeff_dict {(0,): -0.3086, (1,): -0.696, (2,): -0.1554, (3,): -0.5975, (4,): 0.2585, (5,): -0.6503, (0, 1): -0.131, (0, 2): -0.3423, (0, 3): 0.3119, (0, 4): 0.6694, (0, 5): 0.6471, (1, 2): -0.1737, (1, 3): -0.6502, (1, 4): 0.5129, (1, 5): -0.6717, (2, 3): -0.8486, (2, 4): 0.9982, (2, 5): 0.0338, (3, 4): 0.6982, (3, 5): -0.9141, (4, 5): 0.4205}


In [13]:
edge_params_dict, params_init, exp_poss_dict = warm_start_parameters_lightcone(N, tau, Edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)


number of columns is: 6

##################################################
column index 0 column [(0, 1), (2, 3), (4, 5)]

edge (0, 1)

edge (2, 3)

edge (4, 5)

##################################################
column index 1 column [(0, 3), (2, 5)]

edge (0, 3)

edge (2, 5)

##################################################
column index 2 column [(1, 3), (0, 5), (2, 4)]

edge (1, 3)

edge (0, 5)

edge (2, 4)

##################################################
column index 3 column [(1, 5), (0, 4)]

edge (1, 5)

edge (0, 4)

##################################################
column index 4 column [(3, 5), (1, 4), (0, 2)]

edge (3, 5)

edge (1, 4)

edge (0, 2)

##################################################
column index 5 column [(3, 4), (1, 2)]

edge (3, 4)

edge (1, 2)
updated statevector Statevector([0.09288443+0.j, 0.15892892+0.j, 0.06090635+0.j,
             0.10160375+0.j, 0.10631312+0.j, 0.19364802+0.j,
             0.05842584+0.j, 0.12188464+0.j, 0.06044638+0.j,
       

In [14]:
print(exp_poss_dict)
print('\nwarm start fidelity lightcone', list(exp_poss_dict.items())[0])
print('eigen list', eigen_list[:13])
print('eigen list sorted',  np.sort(eigen_list))

{-8.7043: 0.17482526061189915, -7.7197: 0.11479787999154994, -4.7528999999999995: 0.03666916437370985, -4.1201: 0.03782538875097191, -3.7354999999999996: 0.03360673366832242, -3.675099999999999: 0.03749955398502391, -3.621500000000001: 0.037546151382523846, -3.3716999999999997: 0.03643880743654304, -3.2819: 0.025258401079913003, -3.0333: 0.020020902130439635, -2.5727: 0.018016314803741575, -2.0215: 0.021183653172064994, -1.9109000000000003: 0.01512960365375843, -1.8212999999999995: 0.02388372857104718, -1.7686999999999995: 0.018611723327457298, -1.6159000000000003: 0.016270120599957943, -1.5888999999999989: 0.008627516454571504, -1.2745000000000006: 0.014855866429456108, -1.2585000000000002: 0.017224626299570136, -1.2325000000000004: 0.016726282611030478, -1.1799: 0.006173426386091799, -0.6435000000000001: 0.013944068691724652, -0.6128999999999997: 0.011302480526024582, -0.4846999999999998: 0.012472447565528805, -0.33650000000000013: 0.01585550947096257, -0.21770000000000012: 0.0111435

In [15]:
N = 4
tau = 0.2

edges_columns = partition_N(N)
print('edges_columns', edges_columns)
pairs_all = list(itertools.chain.from_iterable(partition_N(N)))
print('pairs_all', pairs_all)
lightcone_dict = find_light_cone(edges_columns)
print('lightcone_dict', lightcone_dict)

# coefficient dictionary just to test 
edge_coeff_dict = {}

edge_coeff_dict.update({(i,): h_val for i, h_val in enumerate(np.random.random(N))})

for keys,values in lightcone_dict.items():
    edge_coeff_dict[keys] = np.random.random()
        
print('edge_coeff_dict' , edge_coeff_dict)

h_list = []
J_list = []

for key,value in edge_coeff_dict.items():
    
    if len(key) == 1:
        h_list.append(value)
    
    else:
        J_list.append(value)

 
print('h_list', h_list)
print('J_list', J_list)

#extrcact Hamiltonian, edge coefficients and eigen list
H = Hamiltonian_qubo(N, pairs_all, h_list, J_list)
eigen_list = H.to_spmatrix().diagonal().real
print('Hamiltonian', H)
print('eigen list', eigen_list[:13])


edges_columns [[(0, 1), (2, 3)], [(0, 3)], [(1, 3), (0, 2)], [(1, 2)]]
pairs_all [(0, 1), (2, 3), (0, 3), (1, 3), (0, 2), (1, 2)]
lightcone_dict {(0, 1): [], (2, 3): [], (0, 3): [(0, 1), (2, 3)], (1, 3): [(0, 3)], (0, 2): [(0, 3)], (1, 2): [(1, 3), (0, 2)]}
edge_coeff_dict {(0,): 0.6896391352767142, (1,): 0.39595254454098816, (2,): 0.3690102388425721, (3,): 0.8124960738950421, (0, 1): 0.6305411740327832, (2, 3): 0.5434085172054177, (0, 3): 0.6020738597621537, (1, 3): 0.19871371855962006, (0, 2): 0.3516208655528683, (1, 2): 0.6213633654462508}
h_list [0.6896391352767142, 0.39595254454098816, 0.3690102388425721, 0.8124960738950421]
J_list [0.6305411740327832, 0.5434085172054177, 0.6020738597621537, 0.19871371855962006, 0.3516208655528683, 0.6213633654462508]
Hamiltonian 0.6896391352767142 * IIIZ
+ 0.39595254454098816 * IIZI
+ 0.3690102388425721 * IZII
+ 0.8124960738950421 * ZIII
+ 0.6305411740327832 * IIZZ
+ 0.5434085172054177 * ZZII
+ 0.6020738597621537 * ZIIZ
+ 0.19871371855962006 * ZI

In [16]:
edge_params_dict, params_list, exp_poss_dict = warm_start_parameters_lightcone(N, tau, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)


number of columns is: 4

##################################################
column index 0 column [(0, 1), (2, 3)]

edge (0, 1)

edge (2, 3)

##################################################
column index 1 column [(0, 3)]

edge (0, 3)

##################################################
column index 2 column [(1, 3), (0, 2)]

edge (1, 3)

edge (0, 2)

##################################################
column index 3 column [(1, 2)]

edge (1, 2)
updated statevector Statevector([0.06118187+0.j, 0.19022274+0.j, 0.15023322+0.j,
             0.25744708+0.j, 0.14796347+0.j, 0.31161269+0.j,
             0.21470835+0.j, 0.25446338+0.j, 0.17920822+0.j,
             0.30387259+0.j, 0.33156264+0.j, 0.32852248+0.j,
             0.26327408+0.j, 0.31732898+0.j, 0.29274813+0.j,
             0.20820373+0.j],
            dims=(2, 2, 2, 2))


In [17]:
print(exp_poss_dict)
print('\nwarm start fidelity lightcone', list(exp_poss_dict.items())[0])
print('eigen list', eigen_list[:13])
print('eigen list sorted',  np.sort(eigen_list))

{-1.9968515766508612: 0.10993378647930073, -1.6972530880173733: 0.09710246563037213, -1.6141415107201524: 0.10792701924153328, -1.4287079189915546: 0.10069767973351142, -1.2380194759304808: 0.09233854913692487, -1.1085700201384046: 0.08570146920623192, -0.6957367510026037: 0.0693132407688645, -0.50390748516278: 0.06627900027605586, -0.38277653526052113: 0.06475161281399465, 0.23632537564591116: 0.04609967614446711, 0.6670694238653716: 0.03618468941892398, 0.6806235080037776: 0.0433487920240685, 0.9014351542699431: 0.03211558655332886, 1.4440135190201921: 0.021893189473171207, 1.5216778879551258: 0.022570021740862305, 5.21481949311441: 0.003743221358388173}

warm start fidelity lightcone (-1.9968515766508612, 0.10993378647930073)
eigen list [ 5.21481949  0.66706942  1.52167789 -0.50390749  1.44401352 -1.69725309
  0.23632538 -0.38277654  0.90143515 -1.23801948 -1.99685158 -1.61414151
 -0.69573675]
eigen list sorted [-1.99685158 -1.69725309 -1.61414151 -1.42870792 -1.23801948 -1.10857002

In [15]:

edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer

q = QuantumRegister(N, name = 'q')
circ = QuantumCircuit(q)
circ.clear()
circ.h(q[::])

tau = 0.2

# Z term
for i in range(N):

    #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
    tauc = tau * edge_coeff_dict[(i,)] 
    para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)

    edge_params_dict[(i,)] = para
    circ.ry(para, i)
    
## ZZ term 

state = Statevector(circ)
updated_state = state 

print('number of columns is:', len(edges_columns))

for column_index, column in enumerate(edges_columns):
    print('\n##################################################')
    print('column index', column_index, 'column', column)

    if column_index == 0: 

        first_column_state = state

        for edge in column:
            
            if len(lightcone_dict[edge]) != 0:
                sys.stderr.write('something is wrong with the lightcones')
                sys.exit()
        
            print('\nedge', edge)  

            tauc = tau * edge_coeff_dict[edge]

            para_init = [0,0]

            final = minimize(square_modulus_cost,
                                para_init,
                                args = (edge, first_column_state, tauc),
                                jac=False,
                                bounds=None,
                                method='L-BFGS-B',
                                callback=None,
                                options={'maxiter': 100})

            para = final.x

            print('opt paramenters', para)
            print('final square modulus', final.fun)

            edge_params_dict[edge] = para
            print('edge_params_dict:', edge_params_dict)

            print('old state', state)
            first_column_state = circuit_update_theta(edge, first_column_state, para)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT

            print('first column state - new state', first_column_state)
        
    else:
        
        for edge in column: 
            if len(lightcone_dict[edge]) == 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    sys.exit()

            print('\nedge', edge)  

            print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

            print('initial state', state)
            
            updated_state = copy.deepcopy(state)

            for old_edge in lightcone_dict[edge]:
                print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge])    
            
            print('updated_state', updated_state)

            edge_list = lightcone_dict[edge] + [edge]
            print('edge list', edge_list)
            
            tauc = tau * edge_coeff_dict[edge]

            para_init = np.zeros(2 + 2*len(lightcone_dict[edge]))
            print('para init', para_init)

            final = minimize(square_modulus_cost_light_cone,
                    para_init,
                    args = (edge_list, state, tauc, updated_state),
                    jac=False,
                    bounds=None,
                    method='L-BFGS-B',
                    callback=None,
                    options={'maxiter': 100})

            para = final.x
            print('opt paramenters', para)
            print('final square modulus', final.fun)

            para = (np.array(para)).reshape(-1, 2)
            print('param list reshaped', para)
            for index, edge in enumerate(edge_list):
                edge_params_dict[edge] = para[index] 
            print('edge_params_dict', edge_params_dict)
        
        print('\n######### previous column update ########')
        print('column index', column_index, 'column', column)
        print('previous column is:', edges_columns[column_index -1])

        for edge in edges_columns[column_index -1]:
            print('edge', edge, 'parameter', edge_params_dict[edge])
            state = circuit_update_theta(edge, state, edge_params_dict[edge])

        print('circuit updated ad the previous column i.e. column', column_index -1)
        print('updated statevector', state)

print('\n######### last column update ########')
print('last column is:', edges_columns[ -1])

for edge in edges_columns[ -1]:
    print('edge', edge, 'parameter', edge_params_dict[edge])
    state = circuit_update_theta(edge, state, edge_params_dict[edge])

print('circuit updated at the last column i.e.', len(edges_columns) - 1)
print('updated statevector', state)

#generate params_list
values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
# Concatenate and flatten all arrays into a single array
flattened_array = np.concatenate(values_as_arrays)
# Convert the flattened array to a list if needed
params_list = flattened_array.tolist()
print(' params_list' , params_list )

NameError: name 'edge_coeff_dict' is not defined